In [16]:
import numpy as np
import pandas as pd
import cv2
import os
import pandas as pd
import time
# import keras.layers as kr

import matplotlib.pyplot as plt
from tqdm import tqdm

In [10]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames.csv')['id'].values

for f in train_path:
    #i = cv2.imread('./train/' + f + '.jpg')
    img = image.load_img('./train/' + f + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train_i.append(x)
for f in test_path:
    #i = cv2.imread('./test/' + f + '.jpg')
    img = image.load_img('./test/' + f + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_i.append(x)

In [3]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import pickle
import random
import os
from tensorflow.keras.models import Model

In [13]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights="imagenet",input_shape=(224, 224, 3) ,include_top=True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [17]:
train_preds = []
test_preds = []

for i in tqdm(train_i):
    #inp = preprocess_input(np.expand_dims(i, axis=0))
    pred = model.predict(i)
    train_preds.append(pred)

for i in tqdm(test_i):
    #inp = preprocess_input(np.expand_dims(i, axis=0))
    pred = model.predict(i)
    test_preds.append(pred)

100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [03:27<00:00,  5.77it/s]


In [18]:
f = open(f'./test_resnet50.pckl','wb')
pickle.dump(test_preds,f)
f.close()

f = open(f'./train_resnet50.pckl','wb')
pickle.dump(train_preds,f)
f.close()

In [43]:
best_train = []
for test in tqdm(test_preds):
    match = [np.sum((train-test)**2)**0.5 for train in train_preds]
    best_match = np.argsort(match)[:3]
    row = train_xy.iloc[best_match]['id'].values
    best_train.append(row)

df = pd.DataFrame(best_train, columns=['3','2','1'])
df.to_csv('resnet50_matches.csv',index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [03:00<00:00,  6.66it/s]


In [119]:
out=[]
for i in range(len(df)):
    coor = train_xy[train_xy['id']== df['1'][i]]
    out.append([test_path[i],coor.iloc[0]['x'],coor.iloc[0]['y']])
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('resnet50_out.csv',index=False)

In [120]:
out=[]
for i in range(len(df)):
    coor = [train_xy[train_xy['id']== df[label][i]] for label in ['1','2','3']]
    result = pd.concat(coor)
    out.append([test_path[i], np.mean(result['x']), np.mean(result['y'])])
    
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('resnet50_top3_out.csv',index=False)

In [9]:
sift = cv2.SIFT_create() # if cv2 version >= 4.4.0 
# sift = cv2.xfeatures2d.SIFT_create() # if cv2 version = 4.3.x 

# Compute SIFT keypoints and descriptors
kp1, des1 = sift.detectAndCompute(des_test[0],None)
kp2, des2 = sift.detectAndCompute(des_test[0],None)

# FLANN parameters and initialize
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)

# Matching descriptor using KNN algorithm
matches = flann.knnMatch(des1,des2,k=2)

# Store all good matches as per Lowe's Ratio test.
ratio = 0.7
good = []
for m,n in matches:
    if m.distance < ratio*n.distance:
        good.append(m)


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\features2d\src\sift.dispatch.cpp:477: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'cv::SIFT_Impl::detectAndCompute'


In [9]:
# SIFT feature matching 
# Initiate SIFT detector
def sift(train_img,test_img):
    sift = cv2.SIFT_create() # if cv2 version >= 4.4.0 
    # sift = cv2.xfeatures2d.SIFT_create() # if cv2 version = 4.3.x 

    # Compute SIFT keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(train_img,None)
    kp2, des2 = sift.detectAndCompute(test_img,None)

    # FLANN parameters and initialize
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)

    # Matching descriptor using KNN algorithm
    matches = flann.knnMatch(des1,des2,k=2)

    # Store all good matches as per Lowe's Ratio test.
    ratio = 0.7
    good = []
    for m,n in matches:
        if m.distance < ratio*n.distance:
            good.append(m)
    
    return len(good)
    # Set a condition that at least MIN_MATCH_NUM matches are required to find the object. 
#     MIN_MATCH_NUM = 4
    

#     if len(good)> MIN_MATCH_NUM:
#         return True
#         # If enough matches are found, we extract the positions of the matched keypoints in both images. 
#         # They are passed to find the perspective transformation. 

#         # Estimate homography between two images
# #         ptsA = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
# #         ptsB = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
# #         H, status = cv2.findHomography(ptsA, 
# #                                        ptsB, 
# #                                        cv2.RANSAC, 
# #                                        ransacReprojThreshold = 5, 
# #                                        maxIters = 500)

# #         matchesMask = status.ravel().tolist()

#         # Draw detected template in scene image
# #         imgOut = cv2.warpPerspective(test_img, H, (train_img.shape[1],train_img.shape[0]),
# #                                  flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

#         # Print total number of successful matches between two images
# #         print("\nNumber of successful matches between two images: ", matchesMask.count(1)) # Returns the number of 1 in the success list

#     else:
#         return False
#         # Otherwise, print that “Not enough matches are found”.
#         print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_NUM) )
#         matchesMask = None


#     # Draw our inliers (if successfully found the object) or all matching keypoints (if failed)
#     draw_params = dict(matchColor = (0,255,0), # draw in a specific colour
#                        singlePointColor = (255,0,0), # draw in a specific colour
#                        matchesMask = matchesMask, # draw only inliers
#                        flags = 2)

#     success_matches = cv2.drawMatches(train_img,kp1,test_img,kp2,good,None,**draw_params)


#     # Plotting results
#     plt.subplots(figsize=(10, 10)) 

#     if matchesMask == None:
#         plt.imshow(success_matches)
#         plt.title('All matching keypoints')
#         plt.axis('off')

#     else:
#         plt.subplot(2,1,1)
#         plt.imshow(success_matches)
#         plt.title('All successful matches')
#         plt.axis('off')

#         plt.subplot(2,1,2)
#         plt.imshow(imgOut, 'gray')
#         plt.title('Display detected template in scene image')
#         plt.axis('off')

#     plt.show()

In [33]:
start = time.time()
matched = []
# for test_name,test_img in test_images:
for train_name,train_img in train_images[::5]:
    score = sift(train_img,test_img)
    if score >= 20:
        matched.append((test_name,train_name))
        end = time.time()
        break
print(end-start)
        

0.5026888847351074


In [20]:

test_image,best_match,score = max(matched,key=lambda x:x[2])
score

93

In [87]:
test_image,best_match,_ = max(matched,key=lambda x:x[2])
coor = train[train['id']== best_match[:-4]]
output = {'id': test_image[:-4], 'x': coor['x'] ,'y': coor['y']}  
output = pd.DataFrame(output,columns= ['id', 'x','y'])  

output.to_csv (r'test.csv', index = False, header=True)